# Edit a pretrained models

Take an existing model that has been trained on many data sets and finish training it on a new dataset.

In [18]:
import keras

### Save new model so it can be loaded later. 

In [43]:
base_model = '/home/wroscoe/models/all_lined_tracks_categorical.h5'
new_model = '/home/wroscoe/models/rally.h5'

###  Train on new data. 

In [38]:
import donkeycar as dk
from donkeycar.parts.keras import KerasCategorical
from donkeycar.parts.datastore import TubGroup

In [39]:
cfg = dk.config.load_config('/home/wroscoe/d2/config.py')

loading config file: /home/wroscoe/d2/config.py
config loaded


### Load base model.

In [40]:
kl = KerasCategorical()
kl.load(base_model)
kl.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
img_in (InputLayer)              (None, 120, 160, 3)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 58, 78, 24)    1824        img_in[0][0]                     
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 27, 37, 32)    19232       conv2d_1[0][0]                   
____________________________________________________________________________________________________
conv2d_3 (Conv2D)                (None, 12, 17, 64)    51264       conv2d_2[0][0]                   
___________________________________________________________________________________________

In [41]:
#Check that the layers are trainable. 
[(l.name, l.trainable) for l in kl.model.layers]

[('img_in', False),
 ('conv2d_1', True),
 ('conv2d_2', True),
 ('conv2d_3', True),
 ('conv2d_4', True),
 ('conv2d_5', True),
 ('flattened', True),
 ('dense_1', True),
 ('dropout_1', True),
 ('dense_2', True),
 ('dropout_2', True),
 ('angle_out', True),
 ('throttle_out', True)]

### Load new dataset

In [42]:
tub_names = ','.join(['/home/wroscoe/data/rally/tub_15_17-11-27/'])

X_keys = ['cam/image_array']
y_keys = ['user/angle', 'user/throttle']

def rt(record):
    record['user/angle'] = dk.utils.linear_bin(record['user/angle'])
    return record

tubgroup = TubGroup(tub_names)

#Create data generators to train the network.
train_gen, val_gen = tubgroup.get_train_val_gen(X_keys, y_keys, record_transform=rt,
                                                batch_size=cfg.BATCH_SIZE,
                                                train_frac=cfg.TRAIN_TEST_SPLIT)


TubGroup:tubpaths: ['/home/wroscoe/data/rally/tub_15_17-11-27']
path_in_tub: /home/wroscoe/data/rally/tub_15_17-11-27
Tub exists: /home/wroscoe/data/rally/tub_15_17-11-27
joining the tubs 1754 records together. This could take 0 minutes.


In [44]:
total_records = len(tubgroup.df)
total_train = int(total_records * cfg.TRAIN_TEST_SPLIT)
total_val = total_records - total_train
print('train: %d, validation: %d' % (total_train, total_val))
steps_per_epoch = total_train // cfg.BATCH_SIZE
print('steps_per_epoch', steps_per_epoch)

kl.train(train_gen,
         val_gen,
         saved_model_path=new_model,
         steps=steps_per_epoch,
         train_split=cfg.TRAIN_TEST_SPLIT)

train: 1403, validation: 351
steps_per_epoch 10
Epoch 1/100
10/10 [==============================] - 12s - loss: 2.0329 - angle_out_loss: 2.2584 - throttle_out_loss: 0.3047 - val_loss: 1.4659 - val_angle_out_loss: 1.6285 - val_throttle_out_loss: 0.2297
Epoch 2/100
10/10 [==============================] - 12s - loss: 1.3155 - angle_out_loss: 1.4614 - throttle_out_loss: 0.2344 - val_loss: 1.2827 - val_angle_out_loss: 1.4250 - val_throttle_out_loss: 0.1694
Epoch 3/100
10/10 [==============================] - 17s - loss: 1.1644 - angle_out_loss: 1.2936 - throttle_out_loss: 0.2170 - val_loss: 1.1829 - val_angle_out_loss: 1.3141 - val_throttle_out_loss: 0.1827
Epoch 4/100
10/10 [==============================] - 14s - loss: 0.9844 - angle_out_loss: 1.0936 - throttle_out_loss: 0.1916 - val_loss: 1.1541 - val_angle_out_loss: 1.2821 - val_throttle_out_loss: 0.1749
Epoch 5/100
10/10 [==============================] - 13s - loss: 0.8586 - angle_out_loss: 0.9537 - throttle_out_loss: 0.1997 - val_l

10/10 [==============================] - 13s - loss: 0.2359 - angle_out_loss: 0.2619 - throttle_out_loss: 0.1165 - val_loss: 1.1544 - val_angle_out_loss: 1.2826 - val_throttle_out_loss: 0.0774
Epoch 23/100
10/10 [==============================] - 13s - loss: 0.2003 - angle_out_loss: 0.2225 - throttle_out_loss: 0.1052 - val_loss: 1.4781 - val_angle_out_loss: 1.6422 - val_throttle_out_loss: 0.0759
Epoch 24/100
10/10 [==============================] - 13s - loss: 0.2104 - angle_out_loss: 0.2336 - throttle_out_loss: 0.1394 - val_loss: 1.0192 - val_angle_out_loss: 1.1324 - val_throttle_out_loss: 0.0830
Epoch 25/100
10/10 [==============================] - 14s - loss: 0.2127 - angle_out_loss: 0.2362 - throttle_out_loss: 0.1072 - val_loss: 1.2298 - val_angle_out_loss: 1.3663 - val_throttle_out_loss: 0.1035
Epoch 26/100
10/10 [==============================] - 13s - loss: 0.1834 - angle_out_loss: 0.2037 - throttle_out_loss: 0.1173 - val_loss: 1.1849 - val_angle_out_loss: 1.3164 - val_throttle_